In [1]:
#!pip install brminer
#!pip install orange3

In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf
import os
import sys
import math
import csv
import seaborn as sns
import matplotlib.pyplot as plt

from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split 
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import Ridge
# the new model
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten


from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import brminer

In [3]:
import numpy as np
import math
import os
import random
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_distances
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class BRM2(BaseEstimator):
    def __init__(self, classifier_count=100, bootstrap_sample_percent=100, use_bootstrap_sample_count=False, bootstrap_sample_count=0, use_past_even_queue=False, max_event_count=3, alpha=0.5, user_threshold=95, distance='euclidean'):
        self.classifier_count = classifier_count
        self.bootstrap_sample_percent = bootstrap_sample_percent
        self.use_bootstrap_sample_count = use_bootstrap_sample_count
        self.bootstrap_sample_count = bootstrap_sample_count
        self.use_past_even_queue = use_past_even_queue
        self.max_event_count = max_event_count
        self.alpha = alpha
        self.user_threshold = user_threshold
        self.distance = distance
        
    def _evaluate(self, current_similarity):
        if (current_similarity < 0):
            current_similarity = 0

        if (self.use_past_even_queue == False):
            return -1+2*current_similarity
        
        result_similarity = (self.alpha * self._similarity_sum / self.max_event_count + (1 - self.alpha) * current_similarity)
        if (result_similarity < 0):
            result_similarity = 0

        self._similarity_sum += current_similarity

        if (len(self._past_events) == self.max_event_count):
            self._similarity_sum -= self._past_events.pop(0)

        self._past_events.append(current_similarity)

        if (self._similarity_sum < 0):
            self._similarity_sum = 0

        return -1+2*result_similarity

    def score_samples(self, X):
        X = np.array(X)
        X_test = self._scaler.transform(X)

        result = []
        batch_size = 100
        for i in range(min(len(X_test), batch_size), len(X_test) + batch_size, batch_size):
            current_X_test = X_test[[j for j in range(max(0, i-batch_size), min(i, len(X_test)))]]

            if self.distance == 'euclidean':
                current_similarity = np.average([np.exp(-np.power(np.amin(euclidean_distances(current_X_test, self._centers[i]), axis=1)/self._max_dissimilarity, 2)/(self._sd[i])) for i in range(len(self._centers))], axis=0)
            
            elif self.distance == 'manhattan':
                current_similarity = np.average([np.exp(-np.power(np.amin(manhattan_distances(current_X_test, self._centers[i]), axis=1)/self._max_dissimilarity, 2)/(self._sd[i])) for i in range(len(self._centers))], axis=0)
            
            elif self.distance == 'cosine':
                current_similarity = np.average([np.exp(-np.power(np.amin(cosine_distances(current_X_test, self._centers[i]), axis=1)/self._max_dissimilarity, 2)/(self._sd[i])) for i in range(len(self._centers))], axis=0)
            
            else:
                print('Distance error')
        
            result = result + [j for j in list(map(self._evaluate, current_similarity))]

        return result
        

    def predict(self, X):
        if (len(X.shape) < 2):
            raise ValueError('Reshape your data')

        if (X.shape[1] != self.n_features_in_):
            raise ValueError('Reshape your data')

        if not self._is_threshold_Computed:            
            x_pred_classif = self.score_samples(self._X_train)            
            x_pred_classif.sort()
            self._inner_threshold = x_pred_classif[(100-self.user_threshold)*len(x_pred_classif)//100]
            self._is_threshold_Computed = True

        y_pred_classif = self.score_samples(X)
        return [-1 if s <= self._inner_threshold else 1 for s in y_pred_classif]
        

    def fit(self, X, y = None):
        # Check that X and y have correct shape
        if y is not None:
            X_train, y_train = check_X_y(X, y)
        else:
             X_train = check_array(X)
             
        self._similarity_sum = 0
        self._is_threshold_Computed = False

        self.n_features_in_ = X_train.shape[1]

        if self.n_features_in_ < 1:
            raise ValueError('Unable to instantiate the train dataset - Empty vector')     
        
        self._scaler = MinMaxScaler()
        X_train = pd.DataFrame(X_train)
        X_train = pd.DataFrame(self._scaler.fit_transform(X_train[X_train.columns]), index=X_train.index, columns=X_train.columns)


        self._max_dissimilarity = math.sqrt(self.n_features_in_)
        self._sd = np.empty(0)
        sampleSize = int(self.bootstrap_sample_count) if (self.use_bootstrap_sample_count) else int(0.01 * self.bootstrap_sample_percent * len(X_train));
        self._centers = np.empty((0, sampleSize, self.n_features_in_))

        list_instances = X_train.values.tolist()
        for i in range(0, self.classifier_count):            
            centers = random.choices(list_instances, k=sampleSize)
            self._centers = np.insert(self._centers, i, centers, axis=0)
            
            if self.distance == 'euclidean':
                self._sd = np.insert(self._sd, i, 2*(np.mean(euclidean_distances(centers, centers))/self._max_dissimilarity)**2)
            
            elif self.distance == 'manhattan':
                self._sd = np.insert(self._sd, i, 2*(np.mean(manhattan_distances(centers, centers))/self._max_dissimilarity)**2)
                
            elif self.distance == 'cosine':
                self._sd = np.insert(self._sd, i, 2*(np.mean(cosine_distances(centers, centers))/self._max_dissimilarity)**2)
                
            else:
                print('Distance Error')
 
        return self

In [4]:
# Function importing Dataset 
def importdata(trainFile, testFile):
    x1 = unpack_csv(trainFile)
    train = pd.read_csv(trainFile, sep= ',', header = None, skiprows=x1) 
    test = pd.read_csv(testFile, sep= ',', header = None, skiprows=x1) 
    return train, test    

# Function to preprocess the dataset 
def preprocess_dataset(train, test): 
    ohe = OneHotEncoder(sparse=True)
    objInTrain = len(train)

    allData = pd.concat([train, test], ignore_index=True, sort =False, axis=0)
    AllDataWihoutClass = allData.iloc[:, :-1]
    AllDataWihoutClassOnlyNominals = AllDataWihoutClass.select_dtypes(include=['object'])
    AllDataWihoutClassNoNominals = AllDataWihoutClass.select_dtypes(exclude=['object'])

    encAllDataWihoutClassNominals = ohe.fit_transform(AllDataWihoutClassOnlyNominals)
    encAllDataWihoutClassNominalsToPanda = pd.DataFrame(encAllDataWihoutClassNominals.toarray())
    
    if AllDataWihoutClassOnlyNominals.shape[1] > 0:
      codAllDataAgain = pd.concat([encAllDataWihoutClassNominalsToPanda, AllDataWihoutClassNoNominals], ignore_index=True, sort =False, axis=1)
    else:
      codAllDataAgain = AllDataWihoutClass

    # Seperating the target variable 
    X_train = codAllDataAgain[:objInTrain]
    y_train = train.values[:, -1]

    X_test = codAllDataAgain[objInTrain:]
    y_test = test.values[:, -1]
    
    return X_train, X_test, y_train, y_test

#Function to split target from data
def splitdataset(train, test): 
    ohe = OneHotEncoder(sparse=True)
    objInTrain = len(train)

    allData = pd.concat([train, test], ignore_index=True, sort =False, axis=0)
    AllDataWihoutClass = allData.iloc[:, :-1]
    AllDataWihoutClassOnlyNominals = AllDataWihoutClass.select_dtypes(include=['object'])
    AllDataWihoutClassNoNominals = AllDataWihoutClass.select_dtypes(exclude=['object'])

    encAllDataWihoutClassNominals = ohe.fit_transform(AllDataWihoutClassOnlyNominals)
    encAllDataWihoutClassNominalsToPanda = pd.DataFrame(encAllDataWihoutClassNominals.toarray())
    
    if AllDataWihoutClassOnlyNominals.shape[1] > 0:
      codAllDataAgain = pd.concat([encAllDataWihoutClassNominalsToPanda, AllDataWihoutClassNoNominals], ignore_index=True, sort =False, axis=1)
    else:
      codAllDataAgain = AllDataWihoutClass

    # Seperating the target variable 
    X_train = codAllDataAgain[:objInTrain]
    y_train = train.values[:, -1]

    X_test = codAllDataAgain[objInTrain:]
    y_test = test.values[:, -1]
    
    mm_scaler = MinMaxScaler()
    X_train_minmax = pd.DataFrame(mm_scaler.fit_transform(X_train[X_train.columns]), index=X_train.index, columns=X_train.columns)
    X_test_minmax = pd.DataFrame(mm_scaler.transform(X_test[X_test.columns]), index=X_test.index, columns=X_test.columns)
    
    std_scaler = StandardScaler()
    X_train_std = pd.DataFrame(std_scaler.fit_transform(X_train[X_train.columns]), index=X_train.index, columns=X_train.columns)
    X_test_std = pd.DataFrame(std_scaler.transform(X_test[X_test.columns]), index=X_test.index, columns=X_test.columns)
    
    X_train_minmax_std = pd.DataFrame(std_scaler.fit_transform(X_train_minmax[X_train_minmax.columns]), index=X_train_minmax.index, columns=X_train_minmax.columns)
    X_test_minmax_std = pd.DataFrame(std_scaler.transform(X_test_minmax[X_test_minmax.columns]), index=X_test_minmax.index, columns=X_test_minmax.columns)
    
    return X_train, X_test, y_train, y_test, X_train_minmax, X_test_minmax, X_train_std, X_test_std, X_train_minmax_std, X_test_minmax_std

# Function to make predictions 
def prediction(X_test, clf_object):  
    y_pred = clf_object.score_samples(X_test) 
    return y_pred

###
def unpack_csv(loc):
    with open(loc, 'r') as myfile:
        data=myfile.read()
        return data.count("@")   

In [5]:
trainFile = 'Unsupervised_Anomaly_Detection\\abalone-17_vs_7-8-9-10\\abalone-17_vs_7-8-9-10-5-1tra.dat'
testFile = 'Unsupervised_Anomaly_Detection\\abalone-17_vs_7-8-9-10\\abalone-17_vs_7-8-9-10-5-1tst.dat'

print('Loading data')
train, test = importdata(trainFile, testFile)

print('Preprocessing data')
X_train, X_test, y_train, y_test = preprocess_dataset(train, test) 

print('Training BRM')
clf = brminer.BRM()
clf.fit(X_train, y_train) 

print('Evaluating BRM')
y_pred = clf.score_samples(X_test)
auc = roc_auc_score(y_test,  y_pred)

print(f'Testing AUC: {auc if auc > .5 else 1 - auc}')

Loading data
Preprocessing data
Training BRM
Evaluating BRM
Testing AUC: 0.830592105263158


In [6]:
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.mixture import GaussianMixture


In [ ]:
folder = 'Unsupervised_Anomaly_Detection'
rootDir = os.getcwd() + '\\' +folder
data = {'folder_name': os.listdir(folder)}

classifiers = [OneClassSVM(), IsolationForest(), GaussianMixture(),
               BRM2(distance='euclidean'), BRM2(distance='manhattan'), BRM2(distance='cosine')]

for clf_classif in classifiers:
    data[str(clf_classif).replace('(distance=\'','').replace('\')','')+'_auc']=[]
    data[str(clf_classif).replace('(distance=\'','').replace('\')','')+'_auc_minmax']=[]
    data[str(clf_classif).replace('(distance=\'','').replace('\')','')+'_auc_std']=[]
    
    for dirName, subdirList, fileList in os.walk(rootDir):
        print('Directorio encontrado: %s' % dirName)
        print("************************************ DIRECTORIO **************************************")
        if len(fileList) > 0: 
            arr_auc = []
            arr_auc_minmax = []
            arr_auc_std = []
            arr_folder_name = dirName.split("/")
            folder_name = arr_folder_name[len(arr_folder_name) - 1]
            completed_name = folder_name + "-5-"

            for i in range(1, int(len(fileList) / 2) + 1):
                print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!") 
                trainFile = dirName + '\\' + fileList[0]
                testFile = dirName + '\\' + fileList[1]

                print('trainFile ' + trainFile)
                print('testFile ' + testFile)
                # Loading data 
                train, test = importdata(trainFile, testFile)

                # Training
                X_train, X_test, y_train, y_test, X_train_minmax, X_test_minmax, X_train_std, X_test_std, X_train_minmax_std, X_test_minmax_std = splitdataset(train, test)

                # Performing training 
                clf_classif.fit(X_train, y_train) 
                # Operational Phase 
                y_pred_classif = prediction(X_test, clf_classif) 

                
                #Training minmax
                clf_classif.fit(X_train_minmax, y_train)
                #Predictt minmax
                y_pred_minmax = prediction(X_test_minmax, clf_classif)
                
                #Training std
                clf_classif.fit(X_train_std, y_train)
                #Predictt std
                y_pred_std = prediction(X_test_std, clf_classif) 
                
                
                auc = metrics.roc_auc_score(y_test,  y_pred_classif)
                arr_auc.append(1 - auc if auc < 0.5 else auc)
                print("AUC: "+str(1 - auc if auc < 0.5 else auc))
                #auc minmax
                auc_minmax = metrics.roc_auc_score(y_test,  y_pred_minmax)
                arr_auc_minmax.append(1 - auc_minmax if auc_minmax < 0.5 else auc_minmax)
                
                #auc minmax
                auc_std = metrics.roc_auc_score(y_test,  y_pred_std)
                arr_auc_std.append(1 - auc_std if auc_std < 0.5 else auc_std)

            print('AUC!! ' + str(arr_auc))
            aver_auc = sum(arr_auc) / len(arr_auc)
            aver_auc_minmax = sum(arr_auc_minmax) / len(arr_auc_minmax)
            aver_auc_std = sum(arr_auc_std) / len(arr_auc_std)
            print('aver_auc!! ' + str(aver_auc))
            
            data[str(clf_classif).replace('(distance=\'','').replace('\')','')+'_auc'].append(aver_auc)
            data[str(clf_classif).replace('(distance=\'','').replace('\')','')+'_auc_minmax'].append(aver_auc_minmax)
            data[str(clf_classif).replace('(distance=\'','').replace('\')','')+'_auc_std'].append(aver_auc_std)

df = pd.DataFrame(data, columns = list(data.keys()))    
df.to_csv('occ_results.csv')

Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection
************************************ DIRECTORIO **************************************
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone-17_vs_7-8-9-10
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone-17_vs_7-8-9-10\abalone-17_vs_7-8-9-10-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone-17_vs_7-8-9-10\abalone-17_vs_7-8-9-10-5-1tst.dat
AUC: 0.5402046783625731
AUC!! [0.5402046783625731]
aver_auc!! 0.5402046783625731
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone-19_vs_10-11-12-13
************************

AUC!! [0.6612903225806451]
aver_auc!! 0.6612903225806451
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-1_vs_2-3-5
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-1_vs_2-3-5\ecoli-0-1_vs_2-3-5-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-1_vs_2-3-5\ecoli-0-1_vs_2-3-5-5-1tst.dat
AUC: 0.8772727272727273
AUC!! [0.8772727272727273]
aver_auc!! 0.8772727272727273
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-1_vs_5
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\

AUC!! [0.6580796252927401]
aver_auc!! 0.6580796252927401
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli4
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli4\ecoli4-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli4\ecoli4-5-1tst.dat
AUC: 0.80078125
AUC!! [0.80078125]
aver_auc!! 0.80078125
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\flare-F
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\flare-F\flare-F-5

AUC: 0.8842378752886836
AUC!! [0.8842378752886836]
aver_auc!! 0.8842378752886836
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-three_vs_eleven
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-three_vs_eleven\kr-vs-k-three_vs_eleven-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-three_vs_eleven\kr-vs-k-three_vs_eleven-5-1tst.dat
AUC: 0.8667698658410732
AUC!! [0.8667698658410732]
aver_auc!! 0.8667698658410732
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-zero-one_vs_draw
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATA

AUC: 0.7328767123287672
AUC!! [0.7328767123287672]
aver_auc!! 0.7328767123287672
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\poker-9_vs_7
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\poker-9_vs_7\poker-9_vs_7-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\poker-9_vs_7\poker-9_vs_7-5-1tst.dat
AUC: 0.925531914893617
AUC!! [0.925531914893617]
aver_auc!! 0.925531914893617
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\segment0
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Des

...

AUC: 0.9941520467836257
AUC!! [0.9941520467836257]
aver_auc!! 0.9941520467836257
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone-3_vs_11
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone-3_vs_11\abalone-3_vs_11-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone-3_vs_11\abalone-3_vs_11-5-1tst.dat
AUC: 0.7074829931972789
AUC!! [0.7074829931972789]
aver_auc!! 0.7074829931972789
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\abalone19
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile

AUC: 0.8445945945945946
AUC!! [0.8445945945945946]
aver_auc!! 0.8445945945945946
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-2-6-7_vs_3-5
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-2-6-7_vs_3-5\ecoli-0-2-6-7_vs_3-5-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-2-6-7_vs_3-5\ecoli-0-2-6-7_vs_3-5-5-1tst.dat
AUC: 0.9
AUC!! [0.9]
aver_auc!! 0.9
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\ecoli-0-3-4-6_vs_5
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\l

AUC: 0.7301136363636364
AUC!! [0.7301136363636364]
aver_auc!! 0.7301136363636364
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\glass-0-1-4-6_vs_2
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\glass-0-1-4-6_vs_2\glass-0-1-4-6_vs_2-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\glass-0-1-4-6_vs_2\glass-0-1-4-6_vs_2-5-1tst.dat
AUC: 0.6351351351351351
AUC!! [0.6351351351351351]
aver_auc!! 0.6351351351351351
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\glass-0-1-5_vs_2
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!

AUC: 0.7967667436489607
AUC!! [0.7967667436489607]
aver_auc!! 0.7967667436489607
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-three_vs_eleven
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-three_vs_eleven\kr-vs-k-three_vs_eleven-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-three_vs_eleven\kr-vs-k-three_vs_eleven-5-1tst.dat
AUC: 0.8043859649122806
AUC!! [0.8043859649122806]
aver_auc!! 0.8043859649122806
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\kr-vs-k-zero-one_vs_draw
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATA

AUC: 0.5842465753424657
AUC!! [0.5842465753424657]
aver_auc!! 0.5842465753424657
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\poker-8_vs_6
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\poker-8_vs_6\poker-8_vs_6-5-1tra.dat
testFile C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\poker-8_vs_6\poker-8_vs_6-5-1tst.dat
AUC: 0.836472602739726
AUC!! [0.836472602739726]
aver_auc!! 0.836472602739726
Directorio encontrado: C:\Users\luisj\Desktop\Tec\Semestre 3\ML\Ass5\Unsupervised_Anomaly_Detection\poker-9_vs_7
************************************ DIRECTORIO **************************************
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! DATASET !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
trainFile C:\Users\luisj

In [ ]:
df.head()

In [ ]:
cols=['OneClassSVM()_auc', 'OneClassSVM()_auc_minmax',
       'OneClassSVM()_auc_std', 'IsolationForest()_auc',
       'IsolationForest()_auc_minmax', 'IsolationForest()_auc_std',
       'GaussianMixture()_auc', 'GaussianMixture()_auc_minmax',
       'GaussianMixture()_auc_std', 'BRM2()_auc', 'BRM2()_auc_minmax',
       'BRM2()_auc_std', 'BRM2manhattan_auc', 'BRM2manhattan_auc_minmax',
       'BRM2manhattan_auc_std', 'BRM2cosine_auc', 'BRM2cosine_auc_minmax',
       'BRM2cosine_auc_std']

df2 = pd.melt(df, id_vars=['folder_name'], value_vars=cols)

In [ ]:
# new data frame with split value columns
new = df2["variable"].str.split("_", n = 2, expand = True)

df2["Classifier"]= new[0].str.replace(r"\(.*\)","")
df2["Metric"]= new[1]
df2["Scaling"]= new[2]
df2.fillna("None",inplace=True)
  
# Dropping old Name columns
#df2.drop(columns =["Name"], inplace = True)

In [ ]:
ranks = df2.groupby("Classifier")["value"].mean().fillna(0).sort_values()[::1].index
plt.figure(figsize=(12,6))
sns.boxplot(x="Classifier", y="value", hue='Scaling', data=df2, order=ranks, palette='Set3')
plt.xticks(rotation = 45)

plt.ylabel('Average AUC')
plt.xlabel('Classifier')
plt.title('AUC score by Classifier')

plt.legend(loc='upper left')
#plt.ylim([0.4, 1.1])

plt.show()

In [ ]:
from scipy.stats import friedmanchisquare

# compare algorithms
stat, p = friedmanchisquare(df.iloc[:,1], df.iloc[:,2], df.iloc[:,3], df.iloc[:,4], df.iloc[:,5],
                           df.iloc[:,6], df.iloc[:,7], df.iloc[:,8], df.iloc[:,9], df.iloc[:,10],
                           df.iloc[:,11], df.iloc[:,12], df.iloc[:,13], df.iloc[:,14], df.iloc[:,15],
                           df.iloc[:,16], df.iloc[:,17], df.iloc[:,18])
print('Statistics=%.3f, p=%.10f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
	print('Same distributions (fail to reject H0)')
else:
	print('Different distributions (reject H0)')

In [ ]:
df3=df
df3 = df3.set_index('folder_name')
df3 = df3.join(df3.rank(axis=1, ascending=False).astype(int).add_suffix('_rank'))

cols= ['OneClassSVM()_auc_rank',
       'OneClassSVM()_auc_minmax_rank', 'OneClassSVM()_auc_std_rank',
       'IsolationForest()_auc_rank', 'IsolationForest()_auc_minmax_rank',
       'IsolationForest()_auc_std_rank', 'GaussianMixture()_auc_rank',
       'GaussianMixture()_auc_minmax_rank', 'GaussianMixture()_auc_std_rank',
       'BRM2()_auc_rank', 'BRM2()_auc_minmax_rank', 'BRM2()_auc_std_rank',
       'BRM2manhattan_auc_rank', 'BRM2manhattan_auc_minmax_rank',
       'BRM2manhattan_auc_std_rank', 'BRM2cosine_auc_rank',
       'BRM2cosine_auc_minmax_rank', 'BRM2cosine_auc_std_rank']

df4 = df3[cols]

av_column = df4.mean(axis=0)
print(av_column)

In [ ]:
import Orange

names = ['OneClassSVM()_auc', 'OneClassSVM()_auc_minmax',
       'OneClassSVM()_auc_std', 'IsolationForest()_auc',
       'IsolationForest()_auc_minmax', 'IsolationForest()_auc_std',
       'GaussianMixture()_auc', 'GaussianMixture()_auc_minmax',
       'GaussianMixture()_auc_std', 'BRM2()_auc', 'BRM2()_auc_minmax',
       'BRM2()_auc_std', 'BRM2manhattan_auc', 'BRM2manhattan_auc_minmax',
       'BRM2manhattan_auc_std', 'BRM2cosine_auc', 'BRM2cosine_auc_minmax',
       'BRM2cosine_auc_std']

avranks =  list(av_column)

cd = Orange.evaluation.compute_CD(avranks, 95) #tested on 30 datasets
print('cd=',cd)

#plt.figure(figsize=(20,8))
Orange.evaluation.graph_ranks(avranks, names, cd=cd, width=18, textspace=1.5)

plt.title('Critical Difference Diagram for Classifiers')
plt.show()